In [55]:
#Imports
import torch
import numpy as np
import torch.nn as nn
import torch.functional as F
import pandas as pd
import os
from torchvision.io import read_image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from PIL import Image
from sklearn.preprocessing import LabelEncoder 

In [56]:
#Constants
img_dir = r"C:\Users\Humperdink2\Documents\github\NewHacks2023\Food Images"
data_dir = r"C:\Users\Humperdink2\Documents\github\NewHacks2023\Food Ingredients and Recipe Dataset with Image Name Mapping.csv"
batch_size = 128
learning_rate = 0.001
momentum = 0.9
img_size = 169

In [57]:
#Processing Dataset:
def getData(img_dir, data_dir):
    """
    Inputs: Image folder directory
            CSV containing labels directory

    Returns: List of labels
             List of image file names
             List of ingredients per recipe
             Number of non-corrupted images in data

    """
    labels = pd.read_csv(data_dir,usecols=[1,4,5])

    count = 0 
    for label in labels.iterrows():
        if count % 1000 == 0:
            print(f"The count is {count}")
        img_path = os.path.join(img_dir, labels.iat[count,1] + ".jpg")
        try:
            image=read_image(img_path)
            if image == None:
                print(count)
                labels = labels.drop(labels.index[count])
                continue
            count += 1
        except:
            print(count)
            labels = labels.drop(labels.index[count])
    labels.reset_index(drop=True, inplace=True)
    return labels.iloc[:,0], labels.iloc[:,1], labels.iloc[:,2], count

# constructing the dataloader
class foodImageDataset(Dataset):
    def __init__(self, img_dir, labels, file_names, ingr, transformation = None):
        self.img_dir = img_dir
        self.ingr = ingr
        self.transform = transformation
        self.labels = labels
        self.file_names = file_names
    """ 
    TO ALTER ADD DATA AUGMENTATION TO THE MODEL
    """
    def __len__(self):
       return len(self.labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.file_names[idx] + ".jpg")

        #Grabbing and closing image
        image = None
        with Image.open(img_path).convert("RGB") as temp:
            image = temp

        label = self.labels[idx]
        if self.transform is not None:
            image = self.transform(image)
        return image, label

    def __getingrs__(self, index):
        return self.ingr[index]

In [ ]:
labels, file_names, ingr, output_size  = getData(img_dir,data_dir)

In [59]:
#Encoding labels
le = LabelEncoder()
targets = le.fit_transform(labels)
targets = torch.as_tensor(targets)

In [60]:
# Model Declaration
class FoodModel(nn.Module):
    def __init__(self,img_size, output_size):
        super().__init__()
        self.model = nn.Sequential(
            #3x169x169
            nn.Conv2d(3,32,kernel_size=(3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)),
            nn.Dropout(0.3),

            #32x84x84
            nn.Conv2d(32,16,kernel_size=(3,3), padding=1),
            nn.MaxPool2d(kernel_size=(2,2)),

            #16x42x42
            nn.Conv2d(16,8,kernel_size=(3,3), padding=1),
            nn.MaxPool2d(kernel_size=(2,2)),

            #8x21*21
            nn.Flatten(),

            #1x(8*21*21)
            nn.Linear(8*21*21, int(output_size*2)),
            nn.ReLU(),
            nn.Dropout(.5),
            
            #1*2700
            nn.Linear(output_size*2, output_size)
        )

    def forward(self, image):
        image = self.model(image)
        return image
    
    def train(self, epochs, train_dl, loss_fn, opt):
        for epoch in range(epochs):
            for image, labels in train_dl:
                labels = labels.to(torch.long)
                preds = self(image)
                print(type(labels))
                loss = loss_fn(preds, labels)
                #training
                opt.zero_grad()
                loss.backward()
                opt.step()
            print(f"epoch {epoch} Completed")

In [61]:
#sending to GPU
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")

def to_device(data, device):
    #Move tensor(s) to chosen device
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.device = device
        self.dl = dl
    
    def __iter__(self):
        for b in self.dl:
            #generating iterater of batches
            yield to_device(b,self.device)
    
    def __len__(self):
        #num batches
        return len(self.dl)

In [67]:
device = get_device()
model = FoodModel(img_size, output_size)
model = to_device(model, device)

In [68]:
#Setting Model
opt = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
loss_fn = nn.CrossEntropyLoss()
transform = transforms.Compose([
    transforms.Resize((169,169)),
    transforms.ToTensor()
])


food = foodImageDataset(img_dir, targets, file_names, ingr, transformation=transform)
food_dl = DataLoader(food, batch_size, shuffle=True)
food_dl = DeviceDataLoader(food_dl, device)

In [ ]:
#Running Model
model.train(5,food_dl,loss_fn,opt)